In [1]:
print('hola')

hola


BUSCAR TODOS LOS LINKS.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [21]:
import requests
import re

# URL de la página
url = 'https://www.skysports.com/premier-league-results/2023-24'

# Hacer la solicitud GET
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    html_content = response.text
else:
    print(f'Error al acceder a la página: {response.status_code}')
    html_content = ""

# Patrón de la expresión regular para extraer los enlaces
pattern = r"http://www\.skysports\.com/football/[a-zA-Z0-9-]+-vs-[a-zA-Z0-9-]+/\d+"

# Buscar todos los enlaces que coincidan con el patrón
links_partidos = re.findall(pattern, html_content)

# Eliminar duplicados si es necesario
links_partidos = list(set(links_partidos))

# Imprimir los enlaces encontrados

print(links_partidos)


['http://www.skysports.com/football/west-ham-united-vs-aston-villa/482879', 'http://www.skysports.com/football/bournemouth-vs-aston-villa/482722', 'http://www.skysports.com/football/aston-villa-vs-everton/482601', 'http://www.skysports.com/football/everton-vs-nottingham-forest/482924', 'http://www.skysports.com/football/crystal-palace-vs-luton-town/482866', 'http://www.skysports.com/football/crystal-palace-vs-nottingham-forest/482664', 'http://www.skysports.com/football/west-ham-united-vs-wolverhampton-wanderers/482760', 'http://www.skysports.com/football/luton-town-vs-chelsea/482785', 'http://www.skysports.com/football/manchester-city-vs-fulham/482629', 'http://www.skysports.com/football/brighton-and-hove-albion-vs-west-ham-united/482614', 'http://www.skysports.com/football/nottingham-forest-vs-arsenal/482805', 'http://www.skysports.com/football/wolverhampton-wanderers-vs-tottenham-hotspur/482710', 'http://www.skysports.com/football/tottenham-hotspur-vs-west-ham-united/482738', 'http:

In [25]:
suma=0
for link in links_partidos:
  suma = suma + 1
print(suma)
#DEBEN SER 380 PARTIDOS

380


Agregamos /stats/ en nuestro enlace.

In [23]:
enlaces_modificados = []
for enlace in links_partidos:
    # Dividir el enlace por partes
    partes = enlace.rsplit('/', 1)  # Divide en la última barra
    if len(partes) == 2:
        nuevo_enlace = f"{partes[0]}/stats/{partes[1]}"
        enlaces_modificados.append(nuevo_enlace)

# Imprimir los enlaces modificados
for enlace in enlaces_modificados:
    print(enlace)

http://www.skysports.com/football/west-ham-united-vs-aston-villa/stats/482879
http://www.skysports.com/football/bournemouth-vs-aston-villa/stats/482722
http://www.skysports.com/football/aston-villa-vs-everton/stats/482601
http://www.skysports.com/football/everton-vs-nottingham-forest/stats/482924
http://www.skysports.com/football/crystal-palace-vs-luton-town/stats/482866
http://www.skysports.com/football/crystal-palace-vs-nottingham-forest/stats/482664
http://www.skysports.com/football/west-ham-united-vs-wolverhampton-wanderers/stats/482760
http://www.skysports.com/football/luton-town-vs-chelsea/stats/482785
http://www.skysports.com/football/manchester-city-vs-fulham/stats/482629
http://www.skysports.com/football/brighton-and-hove-albion-vs-west-ham-united/stats/482614
http://www.skysports.com/football/nottingham-forest-vs-arsenal/stats/482805
http://www.skysports.com/football/wolverhampton-wanderers-vs-tottenham-hotspur/stats/482710
http://www.skysports.com/football/tottenham-hotspur-

Función para extraer todo lo que necesitamos.

In [6]:
# Función para extraer datos de un partido
def extraer_datos_partido(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer los datos, ajusta los selectores según sea necesario

    #Hora y fecha
    hora_y_fecha_detail = soup.find('div', class_='sdc-site-match-header__detail')
    if hora_y_fecha_detail:
        hora_y_fecha_tag = hora_y_fecha_detail.find('time', class_='sdc-site-match-header__detail-time')
        if hora_y_fecha_tag:
            hora_y_fecha = hora_y_fecha_tag.text.split(",")
            reloj = hora_y_fecha[0]
            fecha = hora_y_fecha[1].strip()
        else:
            reloj = 'N/A'
            fecha = 'N/A'
    else:
        reloj = 'N/A'
        fecha = 'N/A'


    #Estadio
    estadio_tag = soup.find('span', class_='sdc-site-match-header__detail-venue')
    if estadio_tag:
      estadio = estadio_tag.text
    else:
      estadio = 'N/A'

    #Asistencia
    asistencia = soup.find('span', class_='sdc-site-match-header__detail-attendance')
    if asistencia:
        asistencia_text = asistencia.get_text(strip=True)
        asistencia = asistencia_text.split()[0]
        asistencia = asistencia.replace('.', '')
        asistencia = asistencia.replace('Attendance:Attendance', '')
    else:
        asistencia = 'N/A'

    #Equipo local y visitante
    equipo_local = soup.find('span', class_="sdc-site-match-header__team-name sdc-site-match-header__team-name--home")
    if equipo_local:
      equipo_local = equipo_local.text.strip()
    else:
      equipo_local= 'N/A'

    equipo_visitante = soup.find('span', class_="sdc-site-match-header__team-name sdc-site-match-header__team-name--away").text
    equipo_visitante = equipo_visitante.strip()

    #Goles local y visitante
    home_score = soup.find('span', {'data-update': 'score-home'})
    if home_score:
      home_score = home_score.text
    else:
      home_score = 'N/A'
    away_score = soup.find('span', {'data-update': 'score-away'})
    if away_score:
      away_score = away_score.text
    else:
      away_score = 'N/A'


    #Posesion
    possession_section = soup.find('h5', string='Possession %').find_next('div', class_='sdc-site-match-stats__stats')
    home_possession = possession_section.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_possession:
      home_possession = home_possession.text
    else:
      home_possession = 'N/A'
    away_possession = possession_section.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_possession:
      away_possession = away_possession.text
    else:
      away_possession = 'N/A'

    #Tiros
    shots_section = soup.find('h5', string='Total Shots').find_next('div', class_='sdc-site-match-stats__stats')
    home_shots = shots_section.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_shots:
      home_shots = home_shots.text
    else:
      home_shots = 'N/A'
    away_shots = shots_section.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_shots:
      away_shots = away_shots.text
    else:
      away_shots = 'N/A'

    #Tiros a puerta
    tiros_puerta = soup.find('h5', string='On Target').find_next('div', class_='sdc-site-match-stats__stats')
    home_on_target_shots = tiros_puerta.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_on_target_shots:
      home_on_target_shots = home_on_target_shots.text
    else:
      home_on_target_shots = 'N/A'
    away_on_target_shots = tiros_puerta.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_on_target_shots:
      away_on_target_shots = away_on_target_shots.text
    else:
      away_on_target_shots = 'N/A'

    #Tiro afuera

    tiros_fuera = soup.find('h5', string='Off Target').find_next('div', class_='sdc-site-match-stats__stats')
    home_off_target_shots = tiros_fuera.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_off_target_shots:
      home_off_target_shots = home_off_target_shots.text
    else:
      home_off_target_shots = 'N/A'
    away_off_target_shots = tiros_fuera.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_off_target_shots:
      away_off_target_shots = away_off_target_shots.text
    else:
      away_off_target_shots = 'N/A'

    #Tiros bloqueados

    tiros_bloqueados = soup.find('h5', string='Blocked').find_next('div', class_='sdc-site-match-stats__stats')
    home_blocked_shots = tiros_bloqueados.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_blocked_shots:
      home_blocked_shots = home_blocked_shots.text
    else:
      home_blocked_shots = 'N/A'
    away_blocked_shots = tiros_bloqueados.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_blocked_shots:
      away_blocked_shots = away_blocked_shots.text
    else:
      away_blocked_shots = 'N/A'


    #Precisión de pases

    pases = soup.find('h5', string='Passing %').find_next('div', class_='sdc-site-match-stats__stats')
    home_passes = pases.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_passes:
      home_passes = home_passes.text
    else:
      home_passes = 'N/A'
    away_passes = pases.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_passes:
      away_passes = away_passes.text
    else:
      away_passes = 'N/A'

    #Oportunidades claras de gol

    oportunidades = soup.find('h5', string='Clear-Cut Chances').find_next('div', class_='sdc-site-match-stats__stats')
    home_op = oportunidades.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_op:
      home_op = home_op.text
    else:
      home_op = 'N/A'
    away_op = oportunidades.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_op:
      away_op = away_op.text
    else:
      away_op = 'N/A'

    #Tiros de esquina

    esquina = soup.find('h5', string='Corners').find_next('div', class_='sdc-site-match-stats__stats')
    home_esquina = esquina.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_esquina:
      home_esquina = home_esquina.text
    else:
      home_esquina = 'N/A'
    away_esquina = esquina.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_esquina:
      away_esquina = away_esquina.text
    else:
      away_esquina = 'N/A'

    #Fuera de lugar

    offside = soup.find('h5', string='Offsides').find_next('div', class_='sdc-site-match-stats__stats')
    home_offside = offside.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_offside:
      home_offside = home_offside.text
    else:
      home_offside = 'N/A'
    away_offside = offside.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_offside:
      away_offside = away_offside.text
    else:
      away_offside = 'N/A'

    #Entradas ganadas

    tackles = soup.find('h5', string='Tackles %').find_next('div', class_='sdc-site-match-stats__stats')
    home_tackles = tackles.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_tackles:
      home_tackles = home_tackles.text
    else:
      home_tackles = 'N/A'
    away_tackles = tackles.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_tackles:
      away_tackles = away_tackles.text
    else:
      away_tackles = 'N/A'

    #Duelos aereos ganados

    fly_tackles = soup.find('h5', string='Aerial Duels %').find_next('div', class_='sdc-site-match-stats__stats')
    home_fly = fly_tackles.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_fly:
      home_fly = home_fly.text
    else:
      home_fly = 'N/A'
    away_fly = fly_tackles.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_fly:
      away_fly = away_fly.text
    else:
      away_fly = 'N/A'



    #Salvadas

    saves = soup.find('h5', string='Saves').find_next('div', class_='sdc-site-match-stats__stats')
    home_saves = saves.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_saves:
      home_saves = home_saves.text
    else:
      home_saves = 'N/A'
    away_saves = saves.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_saves:
      away_saves = away_saves.text
    else:
      away_saves = 'N/A'

    #Faltas

    faltas = soup.find('h5', string='Fouls Committed').find_next('div', class_='sdc-site-match-stats__stats')
    home_faltas = faltas.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_faltas:
      home_faltas = home_faltas.text
    else:
      home_faltas = 'N/A'
    away_faltas = faltas.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_faltas:
      away_faltas = away_faltas.text
    else:
      away_faltas = 'N/A'


    #Amarillas

    amarillas = soup.find('h5', string='Yellow Cards').find_next('div', class_='sdc-site-match-stats__stats')
    home_amarillas = amarillas.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_amarillas:
      home_amarillas = home_amarillas.text
    else:
      home_amarillas = 'N/A'
    away_amarillas = amarillas.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_amarillas:
      away_amarillas = away_amarillas.text
    else:
      away_amarillas = 'N/A'

    #Rojas

    rojas = soup.find('h5', string='Red Cards').find_next('div', class_='sdc-site-match-stats__stats')
    home_rojas = rojas.find('div', class_='sdc-site-match-stats__stats-home').find('span', class_='sdc-site-match-stats__val')
    if home_rojas:
      home_rojas = home_rojas.text
    else:
      home_rojas = 'N/A'
    away_rojas = rojas.find('div', class_='sdc-site-match-stats__stats-away').find('span', class_='sdc-site-match-stats__val')
    if away_rojas:
      away_rojas = away_rojas.text
    else:
      away_rojas = 'N/A'



    # Retornar un diccionario con los datos del partido
    return {
        'Fecha': fecha,
        'Reloj': reloj,
        'Estadio': estadio,
        'Asistencia': asistencia,
        'Equipo Local': equipo_local,
        'Goles Local': home_score,
        'Equipo Visitante': equipo_visitante,
        'Goles Visitante': away_score,
        'Posesión Local': home_possession,
        'Posesión Visitante': away_possession,
        'Tiros Locales Total': home_shots,
        'Tiros Visitante Local': away_shots,
        'Tiros A Puerta Local': home_on_target_shots,
        'Tiros A Puerta Visitante': away_on_target_shots,
        'Tiros Fuera de Objetivo Local': home_off_target_shots,
        'Tiros Fuera de Objetivo Visitante': away_off_target_shots,
        'Tiros Bloqueados Local': home_blocked_shots,
        'Tiros Bloqueados Visitante': away_blocked_shots,
        'Precisión en Pases Local': home_passes,
        'Precisión en Pases Visitante': away_passes,
        'Oportunidades Local': home_op,
        'Oportunidades Visitante': away_op,
        'Esquinas Local': home_esquina,
        'Esquinas Visitante': away_esquina,
        'Fuera de Juego Local': home_offside,
        'Fuera de Juego Visitante': away_offside,
        'Porcentaje de Tackles Ganados Local': home_tackles,
        'Porcentaje de Tackles Ganados Visitante': away_tackles,
        'Porcentaje de Duelo Aéreo Local': home_fly,
        'Porcentaje de Duelo Aéreo Visitante': away_fly,
        'Paradas Local': home_saves,
        'Paradas Visitante': away_saves,
        'Faltas Cometidas Local': home_faltas,
        'Faltas Cometidas Visitante': away_faltas,
        'Amarillas Local': home_amarillas,
        'Amarillas Visitante': away_amarillas,
        'Rojas Local': home_rojas,
        'Rojas Visitante': away_rojas
    }








Prueba de que sirve nuestra función.

In [16]:
url_partido = 'http://www.skysports.com/football/manchester-city-vs-newcastle-united/stats/446646'
datos_partido = extraer_datos_partido(url_partido)

# Mostrar los datos extraídos
print(datos_partido)

{'Fecha': 'Sunday 8th May 2022.', 'Reloj': '4:30pm', 'Estadio': 'Etihad Stadium', 'Asistencia': '53,336', 'Equipo Local': 'Manchester City', 'Goles Local': '5', 'Equipo Visitante': 'Newcastle United', 'Goles Visitante': '0', 'Posesión Local': '71.4', 'Posesión Visitante': '28.6', 'Tiros Locales Total': '21', 'Tiros Visitante Local': '7', 'Tiros A Puerta Local': '9', 'Tiros A Puerta Visitante': '3', 'Tiros Fuera de Objetivo Local': '8', 'Tiros Fuera de Objetivo Visitante': '3', 'Tiros Bloqueados Local': '4', 'Tiros Bloqueados Visitante': '1', 'Precisión en Pases Local': '90.6', 'Precisión en Pases Visitante': '76.9', 'Oportunidades Local': '6', 'Oportunidades Visitante': '3', 'Esquinas Local': '8', 'Esquinas Visitante': '2', 'Fuera de Juego Local': '1', 'Fuera de Juego Visitante': '4', 'Porcentaje de Tackles Ganados Local': '62.5', 'Porcentaje de Tackles Ganados Visitante': '47.4', 'Porcentaje de Duelo Aéreo Local': '63.6', 'Porcentaje de Duelo Aéreo Visitante': '36.4', 'Paradas Local':

Crear la base de datos, extrae los datos necesarios de cada link.

In [24]:
# Lista para almacenar los datos de todos los partidos
datos_partidos = []

# Iterar sobre cada enlace de partido y extraer los datos
for link in enlaces_modificados:
    print(f"Procesando partido: {link}")
    datos_partido = extraer_datos_partido(link)
    datos_partidos.append(datos_partido)

    # Pausa aleatoria entre 2 y 5 segundos para evitar sobrecargar el servidor
    time_to_sleep = random.uniform(2, 5)
    print(f"Esperando {time_to_sleep:.2f} segundos antes de la siguiente solicitud...")
    time.sleep(time_to_sleep)

# Crear un DataFrame con los datos de los partidos
df_partidos = pd.DataFrame(datos_partidos)

# Mostrar los primeros 5 registros
print(df_partidos.head())

# Guardar en un archivo CSV si lo deseas
df_partidos.to_csv('resultados_premier_league_2023_2024.csv', index=False)


Procesando partido: http://www.skysports.com/football/west-ham-united-vs-aston-villa/stats/482879
Esperando 3.56 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/bournemouth-vs-aston-villa/stats/482722
Esperando 2.65 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/aston-villa-vs-everton/stats/482601
Esperando 4.22 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/everton-vs-nottingham-forest/stats/482924
Esperando 2.44 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/crystal-palace-vs-luton-town/stats/482866
Esperando 4.27 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/crystal-palace-vs-nottingham-forest/stats/482664
Esperando 3.65 segundos antes de la siguiente solicitud...
Procesando partido: http://www.skysports.com/football/west-ham-united

In [26]:
df_partidos

,Fecha,Reloj,Estadio,Asistencia,Equipo Local,Goles Local,Equipo Visitante,Goles Visitante,Posesión Local,Posesión Visitante,...,Porcentaje de Duelo Aéreo Local,Porcentaje de Duelo Aéreo Visitante,Paradas Local,Paradas Visitante,Faltas Cometidas Local,Faltas Cometidas Visitante,Amarillas Local,Amarillas Visitante,Rojas Local,Rojas Visitante
0,Sunday 17th March 2024.,2:00pm,London Stadium,"62,461",West Ham United,1,Aston Villa,1,30.6,69.4,...,52.6,47.4,4,3,13,12,3,3,0,0
1,Sunday 3rd December 2023.,2:00pm,Vitality Stadium,"11,191",Bournemouth,2,Aston Villa,2,37.3,62.7,...,47.1,52.9,1,5,16,12,5,3,0,0
2,Sunday 20th August 2023.,2:00pm,Villa Park,"41,694",Aston Villa,4,Everton,0,60.4,39.6,...,55.6,44.4,2,3,8,12,3,4,0,0
3,Sunday 21st April 2024.,1:30pm,Goodison Park,"39,239",Everton,2,Nottingham Forest,0,40.3,59.7,...,54.3,45.7,3,2,15,9,4,2,0,0
4,Saturday 9th March 2024.,3:00pm,Selhurst Park,"25,103",Crystal Palace,1,Luton Town,1,49.5,50.5,...,54.5,45.5,1,3,20,11,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Saturday 11th May 2024.,3:00pm,Goodison Park,"39,221",Everton,1,Sheffield United,0,57.5,42.5,...,58.3,41.7,1,6,6,11,1,2,0,0
376,Saturday 9th March 2024.,12:30pm,Old Trafford,"73,601",Manchester United,2,Everton,0,51.1,48.9,...,33.3,66.7,6,6,13,11,1,2,0,0
377,Friday 18th August 2023.,7:45pm,The City Ground,"29,432",Nottingham Forest,2,Sheffield United,1,49.9,50.1,...,60.8,39.2,2,2,5,9,2,3,0,0
378,Monday 18th September 2023.,7:45pm,The City Ground,"28,958",Nottingham Forest,1,Burnley,1,46.1,53.9,...,54.8,45.2,2,3,16,10,2,3,0,1


In [27]:
nulos_por_columna = df_partidos.isin(['N/A']).sum()
print(nulos_por_columna)

Fecha                                      0
Reloj                                      0
Estadio                                    0
Asistencia                                 8
Equipo Local                               0
Goles Local                                0
Equipo Visitante                           0
Goles Visitante                            0
Posesión Local                             0
Posesión Visitante                         0
Tiros Locales Total                        0
Tiros Visitante Local                      0
Tiros A Puerta Local                       0
Tiros A Puerta Visitante                   0
Tiros Fuera de Objetivo Local              0
Tiros Fuera de Objetivo Visitante          0
Tiros Bloqueados Local                     0
Tiros Bloqueados Visitante                 0
Precisión en Pases Local                   0
Precisión en Pases Visitante               0
Oportunidades Local                        0
Oportunidades Visitante                    0
Esquinas L

In [20]:
nulos_por_columna = df_partidos['Asistencia'].isin(['N/A'])
filas_con_nulos = df_partidos[nulos_por_columna]


print(filas_con_nulos)

                             Fecha   Reloj              Estadio Asistencia  \
41   Saturday 11th September 2021.  3:00pm  St. Mary's Stadium.        N/A   
94   Saturday 18th September 2021.  3:00pm           Turf Moor.        N/A   
189     Saturday 28th August 2021.  5:30pm             Anfield.        N/A   
364       Wednesday 11th May 2022.  8:15pm            Molineux.        N/A   

                Equipo Local Goles Local Equipo Visitante Goles Visitante  \
41               Southampton           0  West Ham United               0   
94                   Burnley           0          Arsenal               1   
189                Liverpool           1          Chelsea               1   
364  Wolverhampton Wanderers           1  Manchester City               5   

    Posesión Local Posesión Visitante  ... Porcentaje de Duelo Aéreo Local  \
41            41.8               58.2  ...                            51.5   
94              45                 55  ...                         